In [23]:
# ! python3.12 -m pip install plotly
# ! python3.12 -m pip install Streamlit
# ! python3.12 -m pip install folium
# ! python3.12 -m pip install fuzzywuzzy
# ! python3.12 -m pip install python-Levenshtein
# ! python3.12 -m pip install pycountry-convert
# ! python3.12 -m pip install streamlit-folium
# ! python3.12 -m pip install branca
# ! python3.12 -m pip install joblib
# ! python3.12 -m pip install pandas
# ! python3.12 -m pip install numpy
# ! python3.12 -m pip install matplotlib
# ! python3.12 -m pip install seaborn
# ! python3.12 -m pip install plotly
# ! python3.12 -m pip install geopandas


In [24]:
# import pandas for structuring the data
import pandas as pd
# import numpy for numerical analysis
import numpy as np

# import libs for diagrams inline with the text
import matplotlib.pyplot as plt
import os
import seaborn as sns

# other utilities
from sklearn import datasets, preprocessing, metrics

# for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm

# for diagramming 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns

# For serialization and deserialization of data from/to file
import pickle

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import folium

In [25]:
import pandas as pd

#Read the CSV files into data frames
dataset_path = os.path.join('DataSæt', 'global air pollution dataset.csv')
dataset_path2 = os.path.join('DataSæt', '2017_-_Cities_Community_Wide_Emissions.csv')

# Loading the datasets
df = pd.read_csv(dataset_path)
df2 = pd.read_csv(dataset_path2)

In [26]:
# Define the correction mapping with the correct capitalization
correction_mapping = {
    "United States of America": "USA",
    "Viet Nam": "Vietnam",
    "Russian Federation": "Russia",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "Bolivia (Plurinational State of)": "Bolivia",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Iran (Islamic Republic of)": "Iran",
    "Syrian Arab Republic": "Syria",
    "Republic of Korea": "South Korea",
    "Lao People's Democratic Republic": "Laos",
    # Add other corrections as needed
}

# Apply the correction mapping to df and df2 and overwrite the original 'Country' column
df['Country'] = df['Country'].replace(correction_mapping).str.strip()
df2['Country'] = df2['Country'].replace(correction_mapping).str.strip()

# Proceed with the merge using the corrected country names
df_merged = pd.merge(df, df2, on='Country', how='inner')

# Rename 'City_x' to 'City' and 'Country_x' to 'Country'
df_merged.rename(columns={'City_x': 'City',}, inplace=True)

# Drop the extra 'Country' column
df_merged.drop(columns=['City_y'], inplace=True)

# Rearrange the columns
column_order = ['Country', 'City', 'AQI Value', 'AQI Category', 'CO AQI Value', 'CO AQI Category', 'Ozone AQI Value', 'Ozone AQI Category', 'NO2 AQI Value', 'NO2 AQI Category', 'PM2.5 AQI Value', 'PM2.5 AQI Category', 'Account number', 'Organization', 'Region', 'C40', 'Access', 'Reporting year', 'Accounting year', 'Boundary', 'Protocol', 'Protocol column', 'Gases included', 'Total emissions (metric tonnes CO2e)', 'Total Scope 1 Emissions (metric tonnes CO2e)', 'Total Scope 2 Emissions (metric tonnes CO2e)', 'Comment', 'Increase/Decrease from last year', 'Reason for increase/decrease in emissions', 'Population', 'Population year', 'GDP', 'GDP Currency', 'GDP Year', 'GDP Source', 'Average annual temperature (in Celsius)​', '​Average altitude (m)', '​Land area (in square km)', 'City Location', 'Country Location']
# Reorder the DataFrame columns
df_merged = df_merged[column_order]


In [27]:
df_merged.drop(columns=['Gases included'], inplace=True)
df_merged.drop(columns=['Protocol column'], inplace=True)
df_merged.drop(columns=['Comment'], inplace=True)
df_merged.drop(columns=['Total Scope 1 Emissions (metric tonnes CO2e)'], inplace=True)
df_merged.drop(columns=['Total Scope 2 Emissions (metric tonnes CO2e)'], inplace=True)
df_merged.drop(columns=['Account number'], inplace=True)
df_merged.drop(columns=['Organization'], inplace=True)
df_merged.drop(columns=['Accounting year'], inplace=True)
df_merged.drop(columns=['Boundary'], inplace=True)
df_merged.drop(columns=['Protocol'], inplace=True)
df_merged.drop(columns=['Increase/Decrease from last year'], inplace=True)
df_merged.drop(columns=['Reason for increase/decrease in emissions'], inplace=True)
df_merged.drop(columns=['Population year'], inplace=True)
df_merged.drop(columns=['GDP Currency'], inplace=True)
df_merged.drop(columns=['GDP Source'], inplace=True)
df_merged.drop(columns=['Access'], inplace=True)



In [28]:
# Convert nulls/NaNs to 'False'
df_merged['C40'] = df_merged['C40'].fillna('False')

# Convert any cell that contains "C40" to 'True', assuming "C40" indicates a true condition
# Adjust the condition as needed to match your data's specific representation of true
df_merged['C40'] = df_merged['C40'].apply(lambda x: 'True' if 'C40' in str(x) else 'False')


In [29]:
# Delete the null values from the data frame
df_merged = df_merged.dropna()

In [30]:
# Extracting latitude and longitude from "City Location" and "Country Location" into new columns
df_merged[['City Latitude', 'City Longitude']] = df_merged['City Location'].str.extract(r'\(([^,]+), ([^)]+)\)')
df_merged[['Country Latitude', 'Country Longitude']] = df_merged['Country Location'].str.extract(r'\(([^,]+), ([^)]+)\)')

# Displaying the first few rows to ensure the transformation was successful

df.head()

,Country,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Russia,Praskoveya,51,Moderate,1,Good,36,Good,0,Good,51,Moderate
1,Brazil,Presidente Dutra,41,Good,1,Good,5,Good,1,Good,41,Good
2,Italy,Priolo Gargallo,66,Moderate,1,Good,39,Good,2,Good,66,Moderate
3,Poland,Przasnysz,34,Good,1,Good,34,Good,0,Good,20,Good
4,France,Punaauia,22,Good,0,Good,22,Good,0,Good,6,Good


In [31]:
# Convert the latitude and longitude columns from strings to floats
df_merged['City Latitude'] = pd.to_numeric(df_merged['City Latitude'], errors='coerce')
df_merged['City Longitude'] = pd.to_numeric(df_merged['City Longitude'], errors='coerce')
df_merged['Country Latitude'] = pd.to_numeric(df_merged['Country Latitude'], errors='coerce')
df_merged['Country Longitude'] = pd.to_numeric(df_merged['Country Longitude'], errors='coerce')

In [32]:
df_merged.drop(columns=['City Location'], inplace=True)
df_merged.drop(columns=['Country Location'], inplace=True)

In [33]:
# Convert 'C40' from strings "True"/"False" to actual booleans
df_merged['C40'] = df_merged['C40'].map({'True': True, 'False': False})

# Create two new columns: 'C40_True' and 'C40_False'
df_merged['C40_True'] = df_merged['C40'].astype(int)  # This will convert True to 1 and False to 0
df_merged['C40_False'] = (~df_merged['C40']).astype(int)  # This inverts the boolean and then converts to 0/1



In [34]:
df_merged.drop(columns=['C40'], inplace=True)

In [35]:
df = df_merged

In [36]:
import pycountry_convert as pc

#applying continent to the dataset for future use of folium mapping
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except:
        return None  # For countries that don't match

# Apply the conversion function to your DataFrame
df['Continent'] = df['Country'].apply(country_to_continent)
# Filter for other continents
north_american_countries_df = df[df['Continent'] == 'North America']
south_american_countries_df = df[df['Continent'] == 'South America']
asian_countries_df = df[df['Continent'] == 'Asia']
african_countries_df = df[df['Continent'] == 'Africa']
oceania_countries_df = df[df['Continent'] == 'Oceania']
Europe_df = df[df['Continent'] == 'Europe']




In [37]:
df = df.drop_duplicates(subset=['City'])


In [38]:
df.count()

Country                                     13412
City                                        13412
AQI Value                                   13412
AQI Category                                13412
CO AQI Value                                13412
CO AQI Category                             13412
Ozone AQI Value                             13412
Ozone AQI Category                          13412
NO2 AQI Value                               13412
NO2 AQI Category                            13412
PM2.5 AQI Value                             13412
PM2.5 AQI Category                          13412
Region                                      13412
Reporting year                              13412
Total emissions (metric tonnes CO2e)        13412
Population                                  13412
GDP                                         13412
GDP Year                                    13412
Average annual temperature (in Celsius)​    13412
​Average altitude (m)                       13412


In [39]:
df.isnull().sum()

Country                                     0
City                                        0
AQI Value                                   0
AQI Category                                0
CO AQI Value                                0
CO AQI Category                             0
Ozone AQI Value                             0
Ozone AQI Category                          0
NO2 AQI Value                               0
NO2 AQI Category                            0
PM2.5 AQI Value                             0
PM2.5 AQI Category                          0
Region                                      0
Reporting year                              0
Total emissions (metric tonnes CO2e)        0
Population                                  0
GDP                                         0
GDP Year                                    0
Average annual temperature (in Celsius)​    0
​Average altitude (m)                       0
​Land area (in square km)                   0
City Latitude                     

In [40]:
df.loc[:, df.isna().any()]
df.loc[288719]
df.fillna(value="Europe", inplace=True)

In [41]:
df.isnull().sum()

Country                                     0
City                                        0
AQI Value                                   0
AQI Category                                0
CO AQI Value                                0
CO AQI Category                             0
Ozone AQI Value                             0
Ozone AQI Category                          0
NO2 AQI Value                               0
NO2 AQI Category                            0
PM2.5 AQI Value                             0
PM2.5 AQI Category                          0
Region                                      0
Reporting year                              0
Total emissions (metric tonnes CO2e)        0
Population                                  0
GDP                                         0
GDP Year                                    0
Average annual temperature (in Celsius)​    0
​Average altitude (m)                       0
​Land area (in square km)                   0
City Latitude                     

In [42]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('string')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13412 entries, 1 to 288719
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Country                                   13412 non-null  string 
 1   City                                      13412 non-null  string 
 2   AQI Value                                 13412 non-null  int64  
 3   AQI Category                              13412 non-null  string 
 4   CO AQI Value                              13412 non-null  int64  
 5   CO AQI Category                           13412 non-null  string 
 6   Ozone AQI Value                           13412 non-null  int64  
 7   Ozone AQI Category                        13412 non-null  string 
 8   NO2 AQI Value                             13412 non-null  int64  
 9   NO2 AQI Category                          13412 non-null  string 
 10  PM2.5 AQI Value                       

In [43]:
df.to_pickle("dataframe.pkl") # save df to a pickle file so it can be used for streamlit